In [169]:
# Célula 1: Importações e configuração inicial
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Certifique-se de ter os recursos do NLTK baixados
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lolop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lolop\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lolop\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [171]:
# Célula 2: Função de limpeza de texto
def clean_text(text, stop_words, stemmer=None, lemmatizer=None):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    if stemmer:
        words = [stemmer.stem(word) for word in words]
    elif lemmatizer:
        words = [lemmatizer.lemmatize(word) for word in words]
    clean_text = ' '.join(words)
    return clean_text


In [173]:
# Célula 3: Função para limpar o CSV
def clean_csv(input_file, output_file, column_to_clean, use_stemming=True):
    df = pd.read_csv(input_file)
    stop_words = set(stopwords.words('portuguese'))
    stemmer = PorterStemmer() if use_stemming else None
    lemmatizer = WordNetLemmatizer() if not use_stemming else None
    df[column_to_clean] = df[column_to_clean].apply(lambda x: clean_text(str(x), stop_words, stemmer, lemmatizer))
    df.to_csv(output_file, index=False)


In [175]:
# Célula 4: Função para adicionar a coluna de fraude
def add_fraud_column(input_file, output_file):
    df = pd.read_csv(input_file)
    # Adiciona uma coluna 'fraude' com base na avaliação (supondo que avaliação 1 e 5 são fraudes)
    df['fraude'] = df['estrelas'].apply(lambda x: 1 if x in [1, 5] else 0)
    df.to_csv(output_file, index=False)


In [191]:
# Célula 5: Função para tokenizar e vetorializar
def tokenize_and_vectorize(input_file, output_file, text_column, use_tfidf=True, max_features=1000):
    df = pd.read_csv(input_file)
    print("Colunas disponíveis:", df.columns)  # Verifique as colunas disponíveis
    if use_tfidf:
        vectorizer = TfidfVectorizer(max_features=max_features)
    else:
        vectorizer = CountVectorizer(max_features=max_features)
    vectors = vectorizer.fit_transform(df[text_column].astype(str))
    vectors_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
    df = pd.concat([df, vectors_df], axis=1)
    df.to_csv(output_file, index=False)


In [179]:
# Célula 6: Função para preparar dados para o modelo
def prepare_data_for_model(input_file, text_column, label_column, max_features=1000):
    df = pd.read_csv(input_file)
    print("Colunas disponíveis:", df.columns)  # Verifique as colunas disponíveis
    vectorizer = TfidfVectorizer(max_features=max_features)
    X_text = vectorizer.fit_transform(df[text_column].astype(str))
    X = pd.DataFrame(X_text.toarray())
    y = df[label_column]
    return X, y


In [181]:
# Célula 7: Função para treinar e avaliar o modelo
def train_and_evaluate_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
    model_rf.fit(X_train, y_train)
    y_pred = model_rf.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    return model_rf


In [193]:
# Célula 8: Limpar o texto e adicionar a coluna de fraude
input_file = 'entrada.csv'  # Substitua pelo caminho do arquivo de entrada
output_file_clean = 'saida.csv'  # Substitua pelo caminho do arquivo limpo
output_file_fraud = 'token.csv'  # Arquivo com a coluna de fraude e tokenização

# Limpar o texto
clean_csv(input_file, output_file_clean, column_to_clean='avaliacao', use_stemming=True)

# Adicionar a coluna de fraude
add_fraud_column(output_file_clean, output_file_fraud)


In [197]:
# Célula 9: Tokenizar e vetorializar (mantendo todas as colunas do arquivo original + tokenização)
tokenize_and_vectorize(output_file_fraud, 'token_completo.csv', text_column='avaliacao', use_tfidf=True, max_features=1000)


Colunas disponíveis: Index(['tempo', 'estrelas', 'avaliacao', 'fraude'], dtype='object')


In [199]:
# Célula 10: Preparar dados para o modelo
X, y = prepare_data_for_model('token_completo.csv', text_column='avaliacao', label_column='fraude')


Colunas disponíveis: Index(['tempo', 'estrelas', 'avaliacao', 'fraude', '10', '200', 'acaba',
       'acho', 'acredito', 'adequado',
       ...
       'valeria', 'vegetariana', 'vegetariano', 'veio', 'vem', 'vergonha',
       'versão', 'vieram', 'voltar', 'ótimo'],
      dtype='object', length=208)


In [202]:
# Célula 11: Treinar e avaliar o modelo
modelo = train_and_evaluate_model(X, y)


Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

